In [1]:
import sys
module_path = "../../src"
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import cv2
import os
import pickle
from detector.detector import detector
from modelo_firmas.inference import firmas_model
from fechas.utils.inference import fecha_model
import config

PATH_DATASET = '../../01.Datasets/'
PATH_IMG_DOCUMENT = os.path.join(PATH_DATASET,"files/reference.png")
PATH_RESULTS_MODEL_FIRMAS = '../../03.SavedModels/modelo_firmas.h5'   

In [2]:
def iterate_prediction(df_test,my_detector,my_firmas_model,my_fechas_model):
    firma1_list = []
    firma2_list = []

    day_list = []
    month_list = []
    year_list = []

    for idx in tqdm(range(len(df_test))):
        id_name = df_test.iloc[idx]['id']
        firma1_class,firma2_class,day,month,year = predict_image(id_name,my_detector,my_firmas_model,my_fechas_model)
        firma1_list.append(firma1_class)
        firma2_list.append(firma2_class)
        day_list.append(day)
        month_list.append(month)
        year_list.append(year)
    df_test['sign_1']= firma1_list
    df_test['sign_2']= firma2_list
    df_test['date_day']= day_list
    df_test['date_month']= month_list
    df_test['date_year']= year_list
    return df_test

In [3]:
def predict_image(id_name,my_detector,my_firmas_model,my_fechas_model):
    imFilename = os.path.join(PATH_DATASET,'images_test',id_name)+'.jpg'
    img = cv2.imread(imFilename)
    img_aligned,fecha,firma1,firma2 = my_detector.predict(img)
    firma1_class,score1 = my_firmas_model.predict(firma1)
    firma2_class,score2 = my_firmas_model.predict(firma2)
    fecha = my_fechas_model.predict_example(fecha)[0]
    day,month,year = fecha[0],fecha[1],fecha[2]
    return firma1_class,firma2_class,day,month,year
    

In [4]:
def main():
    # Loading Models and Process
    data_dict   = open(os.path.join(PATH_DATASET,"data_dict.pkl"), "rb")
    data_dict   = pickle.load(data_dict)

    my_detector     = detector(PATH_IMG_DOCUMENT)
    my_firmas_model = firmas_model(PATH_RESULTS_MODEL_FIRMAS)
    my_fechas_model = fecha_model(config,data_dict)
    df_train = pd.read_csv(os.path.join(PATH_DATASET,'files/output_train.csv'),dtype=str)
    df_test = pd.read_csv(os.path.join(PATH_DATASET,'files/sampleSubmission.csv'),dtype=str)
    submission = iterate_prediction(df_test,my_detector,my_firmas_model,my_fechas_model)
    submission.to_csv('submissions/final_submission.csv',index=False)

In [5]:
if __name__ == '__main__':
    main()

resnet152 does not have coco weights 


/usr/local/lib/python3.6/dist-packages/albumentations/augmentations/transforms.py:1897: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


1it [00:00,  1.87it/s]
1it [00:00,  3.95it/s]
1it [00:00,  3.92it/s]
1it [00:00,  3.87it/s]
1it [00:00,  3.95it/s]
1it [00:00,  3.84it/s]
1it [00:00,  3.35it/s]
1it [00:00,  3.94it/s]
1it [00:00,  3.80it/s]
1it [00:00,  3.22it/s]
1it [00:00,  3.10it/s]
1it [00:00,  3.74it/s]
1it [00:00,  3.89it/s]
1it [00:00,  3.49it/s]
1it [00:00,  3.35it/s]
1it [00:00,  2.93it/s]
1it [00:00,  3.81it/s]
1it [00:00,  3.68it/s]
1it [00:00,  3.90it/s]
1it [00:00,  4.09it/s]
1it [00:00,  3.22it/s]
1it [00:00,  4.04it/s]
1it [00:00,  3.76it/s]
1it [00:00,  3.41it/s]
1it [00:00,  3.39it/s]
1it [00:00,  2.84it/s]
1it [00:00,  3.94it/s]
1it [00:00,  3.39it/s]
1it [00:00,  3.52it/s]
1it [00:00,  3.89it/s]
1it [00:00,  3.89it/s]
1it [00:00,  3.72it/s]
1it [00:00,  3.99it/s]
1it [00:00,  3.85it/s]
1it [00:00,  3.38it/s]
1it [00:00,  3.55it/s]
1it [00:00,  3.70it/s]
1it [00:00,  3.52it/s]
1it [00:00,  3.77it/s]
1it [00:00,  3.74it/s]
1it [00:00,  3.72it/s]
1it [00:00,  3.59it/s]
1it [00:00,  3.58it/s]
1it [00:00,